# Aspect and Sentiment Extraction
Aspect extraction from airy hotel

In [6]:
!pip3 install transformers #python package for transfer learning in NLP
from IPython.display import clear_output #used to clear output
clear_output()

In [7]:
import random
import numpy as np
import pandas as pd
import torch
from torch import optim
from tqdm import tqdm
import os, sys
import string
import re
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer

from transformers import BertConfig, BertTokenizer
from nltk.tokenize import word_tokenize
import itertools
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

In [8]:
from transformers import AlbertPreTrainedModel, BertPreTrainedModel #import pretrained model
import logging #logging purpose
import math
import os

import torch #deep learning library from facebook
from torch import nn #a neural networks library deeply integrated with autograd designed for maximum flexibility
from torch.nn import CrossEntropyLoss, MSELoss

import logging
import math
import os

import torch
from torch import nn
from torch.nn import CrossEntropyLoss, MSELoss
# swish
from transformers.activations import gelu, gelu_new
from transformers import BertConfig
# from transformers.file_utils import add_start_docstrings, add_start_docstrings_to_callable
# from transformers.modeling_utils import PreTrainedModel, prune_linear_layer
# from transformers.modeling_xlm import XLMPreTrainedModel
from transformers import AlbertPreTrainedModel, BertPreTrainedModel, AlbertModel, BertModel, BertConfig, XLMModel, XLMConfig, XLMRobertaModel, XLMRobertaConfig
from transformers import AutoTokenizer, AutoConfig

class BertForWordClassification(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels

        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

        self.init_weights()

    def forward(
        self,
        input_ids=None,
        subword_to_word_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
    ):


        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
        )

        sequence_output = outputs[0]

        # average the token-level outputs to compute word-level representations
        max_seq_len = subword_to_word_ids.max() + 1
        word_latents = []
        for i in range(max_seq_len):
            mask = (subword_to_word_ids == i).unsqueeze(dim=-1)
            word_latents.append((sequence_output * mask).sum(dim=1) / mask.sum())
        word_batch = torch.stack(word_latents, dim=1)

        sequence_output = self.dropout(word_batch)
        logits = self.classifier(sequence_output)

        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here
        if labels is not None:
            loss_fct = CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs

        return outputs  # (loss), scores, (hidden_states), (attentions)

In [9]:
# Forward function for word classification
def forward_word_classification(model, batch_data, i2w, is_test=False, device='cpu', **kwargs):
    # Unpack batch data
    if len(batch_data) == 4:
        (subword_batch, mask_batch, subword_to_word_indices_batch, label_batch) = batch_data
        token_type_batch = None
    elif len(batch_data) == 5:
        (subword_batch, mask_batch, token_type_batch, subword_to_word_indices_batch, label_batch) = batch_data
    
    # Prepare input & label
    subword_batch = torch.LongTensor(subword_batch)
    mask_batch = torch.FloatTensor(mask_batch)
    token_type_batch = torch.LongTensor(token_type_batch) if token_type_batch is not None else None
    subword_to_word_indices_batch = torch.LongTensor(subword_to_word_indices_batch)
    label_batch = torch.LongTensor(label_batch)

    if device == "cuda":
        subword_batch = subword_batch.cuda()
        mask_batch = mask_batch.cuda()
        token_type_batch = token_type_batch.cuda() if token_type_batch is not None else None
        subword_to_word_indices_batch = subword_to_word_indices_batch.cuda()
        label_batch = label_batch.cuda()

    # Forward model
    outputs = model(subword_batch, subword_to_word_indices_batch, attention_mask=mask_batch, token_type_ids=token_type_batch, labels=label_batch)
    loss, logits = outputs[:2]
    
    # generate prediction & label list
    list_hyps = []
    list_labels = []
    hyps_list = torch.topk(logits, k=1, dim=-1)[1].squeeze(dim=-1)
    for i in range(len(hyps_list)):
        hyps, labels = hyps_list[i].tolist(), label_batch[i].tolist()        
        list_hyp, list_label = [], []
        for j in range(len(hyps)):
            if labels[j] == -100:
                break
            else:
                list_hyp.append(i2w[hyps[j]])
                list_label.append(i2w[labels[j]])
        list_hyps.append(list_hyp)
        list_labels.append(list_label)
        
    return loss, list_hyps, list_labels

In [10]:
import sys
import re
from collections import defaultdict, namedtuple
class EvalCounts(object):
    def __init__(self):
        self.correct_chunk = 0    # number of correctly identified chunks
        self.correct_tags = 0     # number of correct chunk tags
        self.found_correct = 0    # number of chunks in corpus
        self.found_guessed = 0    # number of identified chunks
        self.token_counter = 0    # token counter (ignores sentence breaks)

        # counts by type
        self.t_correct_chunk = defaultdict(int)
        self.t_found_correct = defaultdict(int)
        self.t_found_guessed = defaultdict(int)


In [11]:
def start_of_chunk(prev_tag, tag, prev_type, type_):
    # check if a chunk started between the previous and current word
    # arguments: previous and current chunk tags, previous and current types
    chunk_start = False

    if tag == 'B': chunk_start = True
    if tag == 'S': chunk_start = True

    if prev_tag == 'E' and tag == 'E': chunk_start = True
    if prev_tag == 'E' and tag == 'I': chunk_start = True
    if prev_tag == 'S' and tag == 'E': chunk_start = True
    if prev_tag == 'S' and tag == 'I': chunk_start = True
    if prev_tag == 'O' and tag == 'E': chunk_start = True
    if prev_tag == 'O' and tag == 'I': chunk_start = True

    if tag != 'O' and tag != '.' and prev_type != type_:
        chunk_start = True

    # these chunks are assumed to have length 1
    if tag == '[': chunk_start = True
    if tag == ']': chunk_start = True

    return chunk_start

def end_of_chunk(prev_tag, tag, prev_type, type_):
    # check if a chunk ended between the previous and current word
    # arguments: previous and current chunk tags, previous and current types
    chunk_end = False

    if prev_tag == 'E': chunk_end = True
    if prev_tag == 'S': chunk_end = True

    if prev_tag == 'B' and tag == 'B': chunk_end = True
    if prev_tag == 'B' and tag == 'S': chunk_end = True
    if prev_tag == 'B' and tag == 'O': chunk_end = True
    if prev_tag == 'I' and tag == 'B': chunk_end = True
    if prev_tag == 'I' and tag == 'S': chunk_end = True
    if prev_tag == 'I' and tag == 'O': chunk_end = True

    if prev_tag != 'O' and prev_tag != '.' and prev_type != type_:
        chunk_end = True

    # these chunks are assumed to have length 1
    if prev_tag == ']': chunk_end = True
    if prev_tag == '[': chunk_end = True

    return chunk_end

In [12]:
###
# Evaluate Function
###        
def parse_tag(t):
    m = re.match(r'^([^-]*)-(.*)$', t)
    return m.groups() if m else (t, '')

In [13]:
def evaluate_fn(guessed, correct, last_correct, last_correct_type, last_guessed, last_guessed_type, in_correct, counts):
    guessed, guessed_type = parse_tag(guessed)
    correct, correct_type = parse_tag(correct)

    end_correct = end_of_chunk(last_correct, correct,
                               last_correct_type, correct_type)
    end_guessed = end_of_chunk(last_guessed, guessed,
                               last_guessed_type, guessed_type)
    start_correct = start_of_chunk(last_correct, correct,
                                   last_correct_type, correct_type)
    start_guessed = start_of_chunk(last_guessed, guessed,
                                   last_guessed_type, guessed_type)

    if in_correct:
        if (end_correct and end_guessed and
            last_guessed_type == last_correct_type):
            in_correct = False
            counts.correct_chunk += 1
            counts.t_correct_chunk[last_correct_type] += 1
        elif (end_correct != end_guessed or guessed_type != correct_type):
            in_correct = False

    if start_correct and start_guessed and guessed_type == correct_type:
        in_correct = True

    if start_correct:
        counts.found_correct += 1
        counts.t_found_correct[correct_type] += 1
    if start_guessed:
        counts.found_guessed += 1
        counts.t_found_guessed[guessed_type] += 1
    if correct == guessed and guessed_type == correct_type:
        counts.correct_tags += 1
    counts.token_counter += 1

    last_guessed = guessed
    last_correct = correct
    last_guessed_type = guessed_type
    last_correct_type = correct_type
    
    return last_correct, last_correct_type, last_guessed, last_guessed_type, in_correct, counts
    

In [14]:
def evaluate(hyps_list, labels_list):
    counts = EvalCounts()
    num_features = None       # number of features per line
    in_correct = False        # currently processed chunks is correct until now
    last_correct = 'O'        # previous chunk tag in corpus
    last_correct_type = ''    # type of previously identified chunk tag
    last_guessed = 'O'        # previously identified chunk tag
    last_guessed_type = ''    # type of previous chunk tag in corpus

    for hyps, labels in zip(hyps_list, labels_list):
        for hyp, label in zip(hyps, labels):
            
            step_result = evaluate_fn(hyp, label, last_correct, last_correct_type, last_guessed, last_guessed_type, in_correct, counts)
            # print("step result ", step_result)
            last_correct, last_correct_type, last_guessed, last_guessed_type, in_correct, counts = step_result
        # Boundary between sentence
        step_result = evaluate_fn('O', 'O', last_correct, last_correct_type, last_guessed, last_guessed_type, in_correct, counts)
        last_correct, last_correct_type, last_guessed, last_guessed_type, in_correct, counts = step_result
        
    if in_correct:
        counts.correct_chunk += 1
        counts.t_correct_chunk[last_correct_type] += 1

    return counts


In [15]:
Metrics = namedtuple('Metrics', 'tp fp fn prec rec fscore')
def calculate_metrics(correct, guessed, total):
    tp, fp, fn = correct, guessed-correct, total-correct
    p = 0 if tp + fp == 0 else 1.*tp / (tp + fp)
    r = 0 if tp + fn == 0 else 1.*tp / (tp + fn)
    f = 0 if p + r == 0 else (2 * p * r) / (p + r)
    return Metrics(tp, fp, fn, p, r, f)

In [16]:
###
# Calculate Metrics Function
###
def uniq(iterable):
    seen = set()
    return [i for i in iterable if not (i in seen or seen.add(i))]

def metrics(counts):
    c = counts
    overall = calculate_metrics(
        c.correct_chunk, c.found_guessed, c.found_correct
    )
    by_type = {}
    for t in uniq(list(c.t_found_correct.keys()) + list(c.t_found_guessed.keys())):
        by_type[t] = calculate_metrics(
            c.t_correct_chunk[t], c.t_found_guessed[t], c.t_found_correct[t]
        )
      
    return overall, by_type

In [17]:
def conll_evaluation(hyps_list, labels_list):
    counts = evaluate(hyps_list, labels_list)
    c = counts
    overall = calculate_metrics(
        c.correct_chunk, c.found_guessed, c.found_correct
    )
    by_type = {}
    for t in uniq(list(c.t_found_correct.keys()) + list(c.t_found_guessed.keys())):
        by_type[t] = calculate_metrics(
            c.t_correct_chunk[t], c.t_found_guessed[t], c.t_found_correct[t]
        )
    
    c = counts
    acc = c.correct_tags / c.token_counter
    pre = overall.prec
    rec = overall.rec
    f1 = overall.fscore
    
    return (acc, pre, rec, f1)

In [18]:
def aspect_extraction_metrics_fn(list_hyp, list_label):
    metrics = {}
    acc, pre, rec, f1 = conll_evaluation(list_hyp, list_label)
    metrics["ACC"] = acc
    metrics["F1"] = pre
    metrics["REC"] = rec
    metrics["PRE"] = f1
    return metrics

In [19]:

#####
# Term Extraction Airy
#####
class AspectExtractionDataset(Dataset):
    # Static constant variable
    LABEL2INDEX = {'B-ASPECT':0, 'I-ASPECT':1, 'B-SENTIMENT':2, 'I-SENTIMENT':3, 'O':4}
    INDEX2LABEL = {0:'B-ASPECT', 1:'I-ASPECT', 2:'B-SENTIMENT', 3:'I-SENTIMENT', 4:'O'}
    NUM_LABELS = 5
    
    def load_dataset(self, path):
        # Read file
        data = open(path,'r').readlines()

        # Prepare buffer
        dataset = []
        sentence = []
        seq_label = []
        for line in data:
            if '\t' in line:
                token, label = line[:-1].split('\t')
                sentence.append(token)
                seq_label.append(self.LABEL2INDEX[label])
            else:
                dataset.append({
                    'sentence': sentence,
                    'seq_label': seq_label
                })
                sentence = []
                seq_label = []
        return dataset
    
    def __init__(self, dataset_path, tokenizer, *args, **kwargs):
        self.data = self.load_dataset(dataset_path)
        self.tokenizer = tokenizer
        
    def __getitem__(self, index):
        data = self.data[index]
        sentence, seq_label = data['sentence'], data['seq_label']
        
        # Add CLS token
        subwords = [self.tokenizer.cls_token_id]
        subword_to_word_indices = [-1] # For CLS
        
        # Add subwords
        for word_idx, word in enumerate(sentence):
            subword_list = self.tokenizer.encode(word, add_special_tokens=False)
            subword_to_word_indices += [word_idx for i in range(len(subword_list))]
            subwords += subword_list
            
        # Add last SEP token
        subwords += [self.tokenizer.sep_token_id]
        subword_to_word_indices += [-1]
        
        return np.array(subwords), np.array(subword_to_word_indices), np.array(seq_label), data['sentence']
    
    def __len__(self):
        return len(self.data)
       
class AspectExtractionDataLoader(DataLoader):
    def __init__(self, max_seq_len=512, *args, **kwargs):
        super(AspectExtractionDataLoader, self).__init__(*args, **kwargs)
        self.collate_fn = self._collate_fn
        self.max_seq_len = max_seq_len
        
    def _collate_fn(self, batch):
        batch_size = len(batch)
        max_seq_len = max(map(lambda x: len(x[0]), batch))
        max_seq_len = min(self.max_seq_len, max_seq_len)
        max_tgt_len = max(map(lambda x: len(x[2]), batch))
        
        subword_batch = np.zeros((batch_size, max_seq_len), dtype=np.int64)
        mask_batch = np.zeros((batch_size, max_seq_len), dtype=np.float32)
        subword_to_word_indices_batch = np.full((batch_size, max_seq_len), -1, dtype=np.int64)
        seq_label_batch = np.full((batch_size, max_tgt_len), -100, dtype=np.int64)
        
        seq_list = []
        for i, (subwords, subword_to_word_indices, seq_label, raw_seq) in enumerate(batch):
            subwords = subwords[:max_seq_len]
            subword_to_word_indices = subword_to_word_indices[:max_seq_len]
            
            subword_batch[i,:len(subwords)] = subwords
            mask_batch[i,:len(subwords)] = 1
            subword_to_word_indices_batch[i,:len(subwords)] = subword_to_word_indices
            seq_label_batch[i,:len(seq_label)] = seq_label

            seq_list.append(raw_seq)
            
        return subword_batch, mask_batch, subword_to_word_indices_batch, seq_label_batch, seq_list

In [20]:
###
# common functions
###
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
def count_param(module, trainable=False):
    if trainable:
        return sum(p.numel() for p in module.parameters() if p.requires_grad)
    else:
        return sum(p.numel() for p in module.parameters())
    
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def metrics_to_string(metric_dict):
    string_list = []
    for key, value in metric_dict.items():
        string_list.append('{}:{:.2f}'.format(key, value))
    return ' '.join(string_list)

In [21]:
# Set random seed
set_seed(18042021)

# Load IndoBERT Model

In [22]:
# Load Tokenizer and Config
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p1')
config = BertConfig.from_pretrained('indobenchmark/indobert-base-p1')
config.num_labels = AspectExtractionDataset.NUM_LABELS

# Instantiate model
model = BertForWordClassification.from_pretrained('indobenchmark/indobert-base-p1', config=config)

Some weights of BertForWordClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
count_param(model)

124445189

# Prepare Aspect Extraction Dataset

In [24]:
import os
os.chdir('/content')
os.getcwd()

'/content'

In [25]:
!git clone https://github.com/Azhardhiaulhaq/Aspect_Based_Sentiment_Analysis.git

fatal: destination path 'Aspect_Based_Sentiment_Analysis' already exists and is not an empty directory.


In [26]:
train_dataset_path = '/content/Aspect_Based_Sentiment_Analysis/datasets/aspect_sentiment_extraction/train.txt'
valid_dataset_path = '/content/Aspect_Based_Sentiment_Analysis/datasets/aspect_sentiment_extraction/valid.txt'
test_dataset_path = '/content/Aspect_Based_Sentiment_Analysis/datasets/aspect_sentiment_extraction/test.txt'

In [27]:
train_dataset = AspectExtractionDataset(train_dataset_path, tokenizer, lowercase=True)
valid_dataset = AspectExtractionDataset(valid_dataset_path, tokenizer, lowercase=True)
test_dataset = AspectExtractionDataset(test_dataset_path, tokenizer, lowercase=True)

train_loader = AspectExtractionDataLoader(dataset=train_dataset, max_seq_len=512, batch_size=16, num_workers=16, shuffle=True)  
valid_loader = AspectExtractionDataLoader(dataset=valid_dataset, max_seq_len=512, batch_size=16, num_workers=16, shuffle=False)  
test_loader = AspectExtractionDataLoader(dataset=test_dataset, max_seq_len=512, batch_size=16, num_workers=16, shuffle=False)

In [28]:
w2i, i2w = AspectExtractionDataset.LABEL2INDEX, AspectExtractionDataset.INDEX2LABEL
print(w2i)
print(i2w)

{'B-ASPECT': 0, 'I-ASPECT': 1, 'B-SENTIMENT': 2, 'I-SENTIMENT': 3, 'O': 4}
{0: 'B-ASPECT', 1: 'I-ASPECT', 2: 'B-SENTIMENT', 3: 'I-SENTIMENT', 4: 'O'}


# Test model on sample sentences

In [29]:
def word_subword_tokenize(sentence, tokenizer):
    # Add CLS token
    subwords = [tokenizer.cls_token_id]
    subword_to_word_indices = [-1] # For CLS

    # Add subwords
    for word_idx, word in enumerate(sentence):
        subword_list = tokenizer.encode(word, add_special_tokens=False)
        subword_to_word_indices += [word_idx for i in range(len(subword_list))]
        subwords += subword_list

    # Add last SEP token
    subwords += [tokenizer.sep_token_id]
    subword_to_word_indices += [-1]

    return subwords, subword_to_word_indices

In [30]:
import nltk
nltk.download('punkt')
text = word_tokenize('pelayanan	ramah	kamar	nyaman dan fasilitas lengkap hanya	airnya showernya kurang panas')
subwords, subword_to_word_indices = word_subword_tokenize(text, tokenizer)

subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)
subword_to_word_indices = torch.LongTensor(subword_to_word_indices).view(1, -1).to(model.device)
logits = model(subwords, subword_to_word_indices)[0]

preds = torch.topk(logits, k=1, dim=-1)[1].squeeze().numpy()
print(preds)
labels = [i2w[preds[i]] for i in range(len(preds))]

pd.DataFrame({'words': text, 'label': labels})

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[0 4 3 3 3 2 3 3 1 2 3 3]


,words,label
0,pelayanan,B-ASPECT
1,ramah,O
2,kamar,I-SENTIMENT
3,nyaman,I-SENTIMENT
4,dan,I-SENTIMENT
5,fasilitas,B-SENTIMENT
6,lengkap,I-SENTIMENT
7,hanya,I-SENTIMENT
8,airnya,I-ASPECT
9,showernya,B-SENTIMENT


In [31]:
preds

array([0, 4, 3, 3, 3, 2, 3, 3, 1, 2, 3, 3])

In [32]:
text = word_tokenize('tidak	terlalu	jauh dari pusat	kota')
subwords, subword_to_word_indices = word_subword_tokenize(text, tokenizer)

subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)
subword_to_word_indices = torch.LongTensor(subword_to_word_indices).view(1, -1).to(model.device)
logits = model(subwords, subword_to_word_indices)[0]

preds = torch.topk(logits, k=1, dim=-1)[1].squeeze().cpu().numpy()
labels = [i2w[preds[i]] for i in range(len(preds))]

pd.DataFrame({'words': text, 'label': labels})

,words,label
0,tidak,I-SENTIMENT
1,terlalu,I-SENTIMENT
2,jauh,B-ASPECT
3,dari,B-SENTIMENT
4,pusat,B-SENTIMENT
5,kota,B-SENTIMENT


In [33]:
text = word_tokenize('dengan harga	terjangkau kita sudah	mendapatkan	fasilitas	yang nyaman')
subwords, subword_to_word_indices = word_subword_tokenize(text, tokenizer)

subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)
subword_to_word_indices = torch.LongTensor(subword_to_word_indices).view(1, -1).to(model.device)
logits = model(subwords, subword_to_word_indices)[0]

preds = torch.topk(logits, k=1, dim=-1)[1].squeeze().cpu().numpy()
labels = [i2w[preds[i]] for i in range(len(preds))]

pd.DataFrame({'words': text, 'label': labels})

,words,label
0,dengan,B-ASPECT
1,harga,I-SENTIMENT
2,terjangkau,B-ASPECT
3,kita,I-ASPECT
4,sudah,I-SENTIMENT
5,mendapatkan,I-ASPECT
6,fasilitas,B-ASPECT
7,yang,I-SENTIMENT
8,nyaman,B-SENTIMENT


# Fine Tuning & Evaluation

In [34]:
optimizer = optim.Adam(model.parameters(), lr=5e-6)
model = model.cuda()

In [35]:
# Train
n_epochs = 2
for epoch in range(n_epochs):
    model.train()
    torch.set_grad_enabled(True)
 
    total_train_loss = 0
    list_hyp, list_label = [], []

    train_pbar = tqdm(train_loader, leave=True, total=len(train_loader))
    for i, batch_data in enumerate(train_pbar):
        # Forward model
        loss, batch_hyp, batch_label = forward_word_classification(model, batch_data[:-1], i2w=i2w, device='cuda')

        # Update model
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        tr_loss = loss.item()
        total_train_loss = total_train_loss + tr_loss

        # Calculate metrics
        list_hyp += batch_hyp
        list_label += batch_label

        train_pbar.set_description("(Epoch {}) TRAIN LOSS:{:.4f} LR:{:.8f}".format((epoch+1),
            total_train_loss/(i+1), get_lr(optimizer)))

    # Calculate train metric
    metrics = aspect_extraction_metrics_fn(list_hyp, list_label)
    print("(Epoch {}) TRAIN LOSS:{:.4f} {} LR:{:.8f}".format((epoch+1),
        total_train_loss/(i+1), metrics_to_string(metrics), get_lr(optimizer)))

    # Evaluate on validation
    model.eval()
    torch.set_grad_enabled(False)
    
    total_loss, total_correct, total_labels = 0, 0, 0
    list_hyp, list_label = [], []

    pbar = tqdm(valid_loader, leave=True, total=len(valid_loader))
    for i, batch_data in enumerate(pbar):
        batch_seq = batch_data[-1]        
        loss, batch_hyp, batch_label = forward_word_classification(model, batch_data[:-1], i2w=i2w, device='cuda')
        
        # Calculate total loss
        valid_loss = loss.item()
        total_loss = total_loss + valid_loss

        # Calculate evaluation metrics
        list_hyp += batch_hyp
        list_label += batch_label
        metrics = aspect_extraction_metrics_fn(list_hyp, list_label)

        pbar.set_description("VALID LOSS:{:.4f} {}".format(total_loss/(i+1), metrics_to_string(metrics)))
        
    metrics = aspect_extraction_metrics_fn(list_hyp, list_label)
    print("(Epoch {}) VALID LOSS:{:.4f} {}".format((epoch+1),
        total_loss/(i+1), metrics_to_string(metrics)))

  0%|          | 0/52 [00:00<?, ?it/s]

(Epoch 1) TRAIN LOSS:1.0249 ACC:0.76 F1:0.59 REC:0.36 PRE:0.45 LR:0.00000500


VALID LOSS:0.7639 ACC:0.90 F1:0.72 REC:0.81 PRE:0.76: 100%|██████████| 52/52 [00:06<00:00,  8.15it/s]
  0%|          | 0/199 [00:00<?, ?it/s]

(Epoch 1) VALID LOSS:0.7639 ACC:0.90 F1:0.72 REC:0.81 PRE:0.76


  0%|          | 0/52 [00:00<?, ?it/s]

(Epoch 2) TRAIN LOSS:0.7079 ACC:0.91 F1:0.80 REC:0.82 PRE:0.81 LR:0.00000500


VALID LOSS:0.6658 ACC:0.92 F1:0.82 REC:0.89 PRE:0.85: 100%|██████████| 52/52 [00:06<00:00,  8.07it/s]

(Epoch 2) VALID LOSS:0.6658 ACC:0.92 F1:0.82 REC:0.89 PRE:0.85


# Test fine-tuned model with sample sentences

In [ ]:
text = word_tokenize('dengan harga	terjangkau kita sudah	mendapatkan	fasilitas	yang nyaman')
subwords, subword_to_word_indices = word_subword_tokenize(text, tokenizer)

subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)
subword_to_word_indices = torch.LongTensor(subword_to_word_indices).view(1, -1).to(model.device)
logits = model(subwords, subword_to_word_indices)[0]

preds = torch.topk(logits, k=1, dim=-1)[1].squeeze().cpu().numpy()
labels = [i2w[preds[i]] for i in range(len(preds))]

pd.DataFrame({'words': text, 'label': labels})

,words,label
0,dengan,O
1,harga,B-ASPECT
2,terjangkau,B-SENTIMENT
3,kita,O
4,sudah,O
5,mendapatkan,O
6,fasilitas,B-ASPECT
7,yang,O
8,nyaman,B-SENTIMENT


In [ ]:
text = word_tokenize('tidak	terlalu	jauh dari pusat	kota')
subwords, subword_to_word_indices = word_subword_tokenize(text, tokenizer)

subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)
subword_to_word_indices = torch.LongTensor(subword_to_word_indices).view(1, -1).to(model.device)
logits = model(subwords, subword_to_word_indices)[0]

preds = torch.topk(logits, k=1, dim=-1)[1].squeeze().cpu().numpy()
labels = [i2w[preds[i]] for i in range(len(preds))]

pd.DataFrame({'words': text, 'label': labels})

,words,label
0,tidak,B-SENTIMENT
1,terlalu,I-SENTIMENT
2,jauh,O
3,dari,O
4,pusat,O
5,kota,O


In [ ]:
text = word_tokenize('pelayanan	ramah	kamar	nyaman dan fasilitas lengkap hanya	airnya showernya kurang panas')
subwords, subword_to_word_indices = word_subword_tokenize(text, tokenizer)

subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)
subword_to_word_indices = torch.LongTensor(subword_to_word_indices).view(1, -1).to(model.device)
logits = model(subwords, subword_to_word_indices)[0]

preds = torch.topk(logits, k=1, dim=-1)[1].squeeze().cpu().numpy()
labels = [i2w[preds[i]] for i in range(len(preds))]

pd.DataFrame({'words': text, 'label': labels})

,words,label
0,pelayanan,B-ASPECT
1,ramah,B-SENTIMENT
2,kamar,B-ASPECT
3,nyaman,B-SENTIMENT
4,dan,O
5,fasilitas,B-ASPECT
6,lengkap,B-SENTIMENT
7,hanya,O
8,airnya,B-ASPECT
9,showernya,I-ASPECT


In [36]:
# Evaluate on test
model.eval()
torch.set_grad_enabled(False)

total_loss, total_correct, total_labels = 0, 0, 0
list_hyp, list_label = [], []

pbar = tqdm(test_loader, leave=True, total=len(test_loader))
for i, batch_data in enumerate(pbar):
    _, batch_hyp, _ = forward_word_classification(model, batch_data[:-1], i2w=i2w, device='cuda')
    list_hyp += batch_hyp

# Save prediction
df = pd.DataFrame({'label':list_hyp}).reset_index()
df.to_csv('pred.txt', index=False)

print(df)

100%|██████████| 63/63 [00:05<00:00, 11.43it/s]

     index                                              label
0        0  [B-ASPECT, B-SENTIMENT, O, B-ASPECT, B-SENTIME...
1        1                    [B-SENTIMENT, O, O, O, O, O, O]
2        2  [O, B-ASPECT, B-SENTIMENT, O, O, O, B-ASPECT, ...
3        3  [B-ASPECT, B-SENTIMENT, I-SENTIMENT, O, O, O, ...
4        4  [B-ASPECT, B-SENTIMENT, O, B-ASPECT, B-SENTIME...
..     ...                                                ...
995    995  [B-SENTIMENT, O, B-ASPECT, B-SENTIMENT, I-SENT...
996    996  [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...
997    997  [B-ASPECT, B-SENTIMENT, I-SENTIMENT, O, B-SENT...
998    998  [B-ASPECT, B-SENTIMENT, O, B-ASPECT, B-SENTIME...
999    999  [B-ASPECT, B-SENTIMENT, I-SENTIMENT, O, B-ASPE...

[1000 rows x 2 columns]


# DEMO

In [42]:
test_path = '/content/Aspect_Based_Sentiment_Analysis/datasets/aspect_sentiment_extraction/test.txt'

t = 0
f = open(test_path, "r")
list_sentences_test = []
sentence = ""

for x in f:
  sentence = sentence + x.replace("\t", "").replace("\n", "").replace("O", "").replace("B-ASPECT", "").replace("I-ASPECT", "").replace("B-SENTIMENT", "").replace("I-SENTIMENT", "") + " "

  if (x == '\n'):
    # list_sentences_test.append(sentence.strip().replace(" .", ".").replace(" ,", ""))
    list_sentences_test.append(sentence.strip())
    
    sentence = ""

In [53]:
def extract_aspect_sentiment(sentence,label):
  sentence = sentence.split(" ")
  aspect = []
  sentiment = []
  for i in range(len(label)):
    if(label[i]=='B-ASPECT'):
      aspect_text = sentence[i]
      if(label[i+1]!='I-ASPECT'):
        aspect.append(aspect_text)
    elif(label[i]=='I-ASPECT'):
      if(label[i-1]!='B-ASPECT'):
        aspect_text = sentence[i]
      else :
        aspect_text = aspect_text +' ' + sentence[i]
      if (i+1 == len(label)):
        aspect.append(aspect_text)
      elif (label[i+1]!='I-ASPECT'):
        aspect.append(aspect_text)
    elif(label[i]=='B-SENTIMENT'):
      sentiment_text = sentence[i]
      if(label[i+1]!='I-SENTIMENT'):
        sentiment.append(sentiment_text)
    elif(label[i]=='I-SENTIMENT'):
      if(label[i-1]!='B-SENTIMENT'):
        sentiment_text = sentence[i]
      else :
        sentiment_text = sentiment_text +' ' + sentence[i]
      if (i+1 == len(label)):
        sentiment.append(sentiment_text)
      elif (label[i+1]!='I-SENTIMENT'):
        sentiment.append(sentiment_text)
  return aspect, sentiment

In [65]:
sentence = list_sentences_test[5]
aspect_sentiment_extraction = list_hyp[5]

In [66]:
aspect, sentiment = extract_aspect_sentiment(sentence, aspect_sentiment_extraction)

relation = [x + ' ' + y for x in aspect for y in sentiment] 

In [69]:
list_sentence_demo = []
for item in relation:
  list_sentence_demo.append(sentence+" "+item)

In [ ]:
def print_polarity(sentence,sentiment):
    sent = sentence[0].split('.')[:-1]
    relation = []
    for sentc in sentence : 
        temp_relation = sentc.split('.')[-1]
        relation.append(temp_relation)
    print("Untuk kalimat review berikut : ")
    print(sent[0])
    print("\n")
    print("{:25}{:5}".format("Relation","Polarity"))
    print("-" *30)
    for rel, pol in zip(relation,sentiment):
        print("{:25}{:5}".format(rel,pol))

In [2]:
import warnings
warnings.filterwarnings('ignore')

from Aspect_Based_Sentiment_Analysis.modules import PolarityDetection

polarity_detection = PolarityDetection.PolarityDetection()
model_polarity_path = "/content/Aspect_Based_Sentiment_Analysis/modules/model/PolarityDetection/"
tokenizer_polarity_path = "/content/Aspect_Based_Sentiment_Analysis/modules/model/tokenizer_polarity.pickle"
encoder_polarity_path = "/content/Aspect_Based_Sentiment_Analysis/modules/model/encoder_polarity.pickle"

polarity_detection.load_model(model_polarity_path, tokenizer_polarity_path, encoder_polarity_path)

sentiment = polarity_detection.predict(["adsfasdf"])

print_polarity(sentence, sentiment)

['Positive']

In [3]:
from Aspect_Based_Sentiment_Analysis.modules import RelationClassification

relation_classification = RelationClassification.RelationClassification()
model_relation_path = "/content/Aspect_Based_Sentiment_Analysis/modules/model/RelationClassification/"
tokenizer_relation_path = "/content/Aspect_Based_Sentiment_Analysis/modules/model/tokenizer_relation.pickle"
encoder_relation_path = "/content/Aspect_Based_Sentiment_Analysis/modules/model/encoder_relation.pickle"

relation_classification.load_model(model_relation_path, tokenizer_relation_path, encoder_relation_path)

relation_classification.predict(["adsfasdf"])

ValueError: ignored